# PODCAST SUMMARIZER USING PYTHON  SCRIPT

Importing libraries


In [2]:
#importing libraries
import pandas as pd
import transformers
import openai
from transformers import BartTokenizer, BartForConditionalGeneration
import os

Parsing our data and reading the first five rows

In [11]:
#specifying file path
text_data = pd.read_csv("116_diarized_timestamped_transcript.txt", delimiter="\t")
text_data

,"Speaker 0 (00:00:00,039-00:00:06,457): Podcast. Take it out. The Joe, Regan. Experience. Train my day, Joe Regan podcast, my day."
0,"Speaker 1 (00:00:11,273-00:00:13,492): George,..."
1,"Speaker 0 (00:00:14,959-00:00:15,399): Thank you."
2,"Speaker 1 (00:00:15,399-00:00:16,579): What do..."
3,"Speaker 0 (00:00:16,878-00:00:21,456): Yeah. L..."
4,"Speaker 1 (00:00:21,916-00:00:24,455): Does it..."
...,...
1013,"Speaker 0 (01:57:11,752-01:57:26,583): I reall..."
1014,"Speaker 1 (01:57:26,583-01:57:31,260): Well, G..."
1015,"Speaker 0 (01:57:31,260-01:57:31,580): you, sir."
1016,"Speaker 1 (01:57:31,580-01:57:34,619): Susan. ..."


In [14]:
import nltk
from nltk.tokenize import sent_tokenize

# Download necessary NLTK resources if not already present
nltk.download('punkt')

# Define the transcript text
transcript = "116_diarized_timestamped_transcript.txt"

# Tokenize the transcript into sentences
sentences = sent_tokenize(transcript)

# Define a function to identify and summarize sections by speaker
def summarize_by_speaker(sentences):
    speakers = {}
    current_speaker = None
    summary = ""
    for sentence in sentences:
        # Identify speaker based on name at the beginning of the sentence
        if sentence.startswith("Speaker"):
            speaker, _ = sentence.split(":", 1)
            current_speaker = speaker.strip()
            speakers[current_speaker] = []
        else:
            #Add sentence to the current speaker's summary
            speakers[current_speaker].append(sentence)

    # Generate summaries for each speaker
    for speaker, speaker_sentences in speakers.items():
        speaker_summary = " ".join(speaker_sentences)
        summary += f"\n**{speaker}** (00:00:00,000-00:00:00,000): {speaker_summary}"
    return summary

# Generate the summary
summary = summarize_by_speaker(sentences)

# Write the summary to a text file
with open("podcast_summary.txt", "w") as f:
    f.write(summary)

print("Summary written to podcast_summary.txt")


[nltk_data] Downloading package punkt to /home/scientist-
[nltk_data]     ezzy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


KeyError: None